In [ ]:
import os
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, text
import time
import numpy as np
import dotenv

dotenv.load_dotenv(".env.local")

from test_unit.tasks_unit1 import UnitTask1
from mrp_functions.base_tasks import TaskBase
from mrp_functions import logging
import re

np.__version__

part_group=os.environ['PART_GROUP']
contract_code=os.environ['CONTRACT_CODE']
mrp_run_date=os.environ['MRP_RUN_DATE']
msc_code=os.environ['MSC_CODE']
plant_code=os.environ['PLANT_CODE']
production_plan_id=int(os.environ['PRODUCTION_PLAN_ID'])
user_code=int(os.environ['USER_CODE'])
simulation=bool(int(os.environ['SIMULATION']))

tb = TaskBase()
tb.db_connection_health_check()

In [ ]:
# Get end date of production plan
end_date_of_prod_plan = tb.query_end_date_from_production_plan(production_plan_id)
if end_date_of_prod_plan is None:
    raise ValueError("Production Plan End Date is None!")
# Get list of all date from N to N+6 to end of production Plan
list_mrp_date = tb.query_list_date_from_Nplus1_to_Nplus6_definition(mrp_run_date, end_date_of_prod_plan)
print(f"Inventory Forecast From: {list_mrp_date[0]} To: {list_mrp_date[-1]}")


In [ ]:
# Get list of order which has ETA in that time
list_order_eta_in_chunk = tb.query_current_logical_inventory_w_order_list_eta(mrp_run_date, plant_code)
# Convert order to Logical Inventory Forecast and Upsert to Logical Inventory
for list_order_eta in list_order_eta_in_chunk:
    list_order_eta = list_order_eta.set_index(['part_code', 'part_color_code'])
    break

list_order_eta = list_order_eta.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')
list_order_eta

In [ ]:
sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                       index=list_mrp_date)
sim_logical_parts_inv.index.names = ['production_date']
sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                            names=['part_code', 'part_color_code'])
sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)
sim_logical_parts_inv

In [ ]:
sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

sim_logical_parts_inv['plant_code'] = plant_code
sim_logical_parts_inv['created_by'] = user_code
sim_logical_parts_inv['updated_by'] = user_code
sim_logical_parts_inv

In [ ]:
# Get list of order which has ETA in that time
list_order_eta_in_chunk = tb.query_current_logical_inventory_w_order_list_eta(mrp_run_date, plant_code)
# Convert order to Logical Inventory Forecast and Upsert to Logical Inventory
for list_order_eta in list_order_eta_in_chunk:
    list_order_eta = list_order_eta.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')

    sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                           index=list_mrp_date)
    sim_logical_parts_inv.index.names = ['production_date']
    sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                                names=['part_code', 'part_color_code'])
    sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)

    sim_logical_parts_inv = sim_logical_parts_inv.loc[sim_logical_parts_inv.index > datetime.date(datetime.strptime(mrp_run_date, "%Y-%m-%d"))]
    sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
    sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
    sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

    sim_logical_parts_inv['plant_code'] = plant_code
    sim_logical_parts_inv['created_by'] = user_code
    sim_logical_parts_inv['updated_by'] = user_code
    # tb.store_logical_inventory_results(sim_logical_parts_inv.to_dict(orient="records"),
    #                                    upsert=True, is_simulation=simulation)
sim_logical_parts_inv

# FINISH PREPARE LOGICAL INVENTORY WITH FUTURE ORDER ETA !


In [ ]:
sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                       index=list_mrp_date)
sim_logical_parts_inv.index.names = ['production_date']
sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                            names=['part_code', 'part_color_code'])
sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)
sim_logical_parts_inv

In [ ]:
sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

sim_logical_parts_inv['plant_code'] = plant_code
sim_logical_parts_inv['created_by'] = user_code
sim_logical_parts_inv['updated_by'] = user_code
sim_logical_parts_inv

In [ ]:
# Get list of order which has ETA in that time
list_order_eta_in_chunk = tb.query_current_logical_inventory_w_order_list_eta(mrp_run_date, plant_code)
# Convert order to Logical Inventory Forecast and Upsert to Logical Inventory
for list_order_eta in list_order_eta_in_chunk:
    list_order_eta = list_order_eta.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')

    sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                           index=list_mrp_date)
    sim_logical_parts_inv.index.names = ['production_date']
    sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                                names=['part_code', 'part_color_code'])
    sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)

    sim_logical_parts_inv = sim_logical_parts_inv.loc[sim_logical_parts_inv.index > datetime.date(datetime.strptime(mrp_run_date, "%Y-%m-%d"))]
    sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
    sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
    sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

    sim_logical_parts_inv['plant_code'] = plant_code
    sim_logical_parts_inv['created_by'] = user_code
    sim_logical_parts_inv['updated_by'] = user_code
    # tb.store_logical_inventory_results(sim_logical_parts_inv.to_dict(orient="records"),
    #                                    upsert=True, is_simulation=simulation)
sim_logical_parts_inv

# FINISH PREPARE LOGICAL INVENTORY WITH FUTURE ORDER ETA !


In [ ]:
sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                       index=list_mrp_date)
sim_logical_parts_inv.index.names = ['production_date']
sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                            names=['part_code', 'part_color_code'])
sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)
sim_logical_parts_inv

In [ ]:
sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

sim_logical_parts_inv['plant_code'] = plant_code
sim_logical_parts_inv['created_by'] = user_code
sim_logical_parts_inv['updated_by'] = user_code
sim_logical_parts_inv

In [ ]:
# Get list of order which has ETA in that time
list_order_eta_in_chunk = tb.query_current_logical_inventory_w_order_list_eta(mrp_run_date, plant_code)
# Convert order to Logical Inventory Forecast and Upsert to Logical Inventory
for list_order_eta in list_order_eta_in_chunk:
    list_order_eta = list_order_eta.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')

    sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                           index=list_mrp_date)
    sim_logical_parts_inv.index.names = ['production_date']
    sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                                names=['part_code', 'part_color_code'])
    sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)

    sim_logical_parts_inv = sim_logical_parts_inv.loc[sim_logical_parts_inv.index > datetime.date(datetime.strptime(mrp_run_date, "%Y-%m-%d"))]
    sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
    sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
    sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

    sim_logical_parts_inv['plant_code'] = plant_code
    sim_logical_parts_inv['created_by'] = user_code
    sim_logical_parts_inv['updated_by'] = user_code
    # tb.store_logical_inventory_results(sim_logical_parts_inv.to_dict(orient="records"),
    #                                    upsert=True, is_simulation=simulation)
sim_logical_parts_inv

# FINISH PREPARE LOGICAL INVENTORY WITH FUTURE ORDER ETA !


In [ ]:
sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                       index=list_mrp_date)
sim_logical_parts_inv.index.names = ['production_date']
sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                            names=['part_code', 'part_color_code'])
sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)
sim_logical_parts_inv

In [ ]:
sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

sim_logical_parts_inv['plant_code'] = plant_code
sim_logical_parts_inv['created_by'] = user_code
sim_logical_parts_inv['updated_by'] = user_code
sim_logical_parts_inv

In [ ]:
# Get list of order which has ETA in that time
list_order_eta_in_chunk = tb.query_current_logical_inventory_w_order_list_eta(mrp_run_date, plant_code)
# Convert order to Logical Inventory Forecast and Upsert to Logical Inventory
for list_order_eta in list_order_eta_in_chunk:
    list_order_eta = list_order_eta.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')

    sim_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(list_order_eta.columns))),
                                           index=list_mrp_date)
    sim_logical_parts_inv.index.names = ['production_date']
    sim_logical_parts_inv.columns = pd.MultiIndex.from_tuples(list_order_eta.columns,
                                                                names=['part_code', 'part_color_code'])
    sim_logical_parts_inv = (sim_logical_parts_inv + list_order_eta).fillna(0.0)

    sim_logical_parts_inv = sim_logical_parts_inv.loc[sim_logical_parts_inv.index > datetime.date(datetime.strptime(mrp_run_date, "%Y-%m-%d"))]
    sim_logical_parts_inv = sim_logical_parts_inv.stack().reset_index()
    sim_logical_parts_inv = sim_logical_parts_inv.set_index(['production_date', 'part_color_code']).stack().reset_index()
    sim_logical_parts_inv.rename(columns={sim_logical_parts_inv.columns[-1]: 'quantity'}, inplace=True)

    sim_logical_parts_inv['plant_code'] = plant_code
    sim_logical_parts_inv['created_by'] = user_code
    sim_logical_parts_inv['updated_by'] = user_code
    # tb.store_logical_inventory_results(sim_logical_parts_inv.to_dict(orient="records"),
    #                                    upsert=True, is_simulation=simulation)
sim_logical_parts_inv

# FINISH PREPARE LOGICAL INVENTORY WITH FUTURE ORDER ETA !
